<a href="https://colab.research.google.com/github/mansfire/model_1_signal/blob/main/examplemodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from matplotlib import pyplot
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import explained_variance_score, mean_squared_error, r2_score, mean_absolute_error
from collections import Counter
from keras import callbacks
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout,InputLayer
from tensorflow.keras.models import Sequential
from keras.layers import BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras import backend
from random import shuffle
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive



In [ ]:

  #initialize and gather data
drive.mount('/content/drive')
data = pd.read_excel('/content/drive/MyDrive/New_data.xlsx')
data=data.drop(['Unnamed: 0'],axis=1)

data.astype(np.float32)


  #split the data
    #again, ke is exit as we want to see how we can prevent this from occuring
  ##Splitting Independent and dependent variable in X and Y respectively
X = data.drop(['signal_after','path_loss'],axis=1)
Y = data[['path_loss','signal_after']].values

print(X.shape)
print(Y.shape)
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.1)
X_train = X_train.astype(np.float64)

y_train = y_train.astype(np.float64)


  # build the model!
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
model = Sequential()
model.add(Dense(1000, input_shape=(X_train.shape[1],), activation='relu')) # (features,)
model.add(Dropout(0.2))
model.add(Dense(750, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(150, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(75, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(20, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='linear')) # output node
model.summary() # see what your model looks like

  # compile the model
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])

  # early stopping callback
es = EarlyStopping(monitor='val_loss',
                  mode='min',
                  patience=1000,
                  restore_best_weights = True)

  # fit the model!
  # attach it to a new variable called 'history' in case
  # to look at the learning curves
history = model.fit(X_train, y_train,
                    validation_data = (X_test, y_test),
                    callbacks=[es],
                    epochs=5000,
                    batch_size=50,
                    verbose=1)
pred = model.predict(X_test)
pred

trainpreds = model.predict(X_train)

from sklearn.metrics import mean_absolute_error
pred_mae=mean_absolute_error(y_train, trainpreds)
print(mean_absolute_error(y_train, trainpreds)) # train
print(mean_absolute_error(y_test, pred)) # test
out_preds=(pred,trainpreds)

  #Read in the data from excel (I converted form csv)
data2 = pd.read_excel('/content/drive/MyDrive/room1.xlsx')
data2=data2.drop(['Router','Cell'],axis=1)

data2.astype(np.float32)
data2=data2.align(X_train,axis=1)[0]

pred2 = model.predict(data2)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(538, 53)
(538, 2)
Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_88 (Dense)            (None, 1000)              54000     
                                                                 
 dropout_70 (Dropout)        (None, 1000)              0         
                                                                 
 dense_89 (Dense)            (None, 750)               750750    
                                                                 
 dropout_71 (Dropout)        (None, 750)               0         
                                                                 
 dense_90 (Dense)            (None, 500)               375500    
                                                                 
 dropout_72 (Dropout)        (None, 5

In [ ]:
idx=pred2.argmin()
cell_x=data2['cell_x'][idx]
best_x=cell_x
cell_y=data2['cell_y'][idx]
best_y=cell_y
cell_z=data2['cell_z'][idx]

best_z=cell_z

KeyError: ignored

In [ ]:
!pip install h5py


In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")


Saved model to disk


In [ ]:
Predictors=['router_x',	'router_y',	'router_z',	'cell_x',	'cell_y',	'cell_z',
            'pass_0',	'pass_dis_0',	'pass_type_0',	'pass_mat_0',
            'pass_1',	'pass_dis_1',	'pass_type_1',	'pass_mat_1',
            'pass_2',	'pass_dis_2',	'pass_type_2',	'pass_mat_2',
            'pass_3',	'pass_dis_3',	'pass_type_3',	'pass_mat_3',
            'pass_4',	'pass_dis_4',	'pass_type_4',	'pass_mat_4',
            'pass_5',	'pass_dis_5',	'pass_type_5',	'pass_mat_5',
            'pass_6',	'pass_dis_6',	'pass_type_6',	'pass_mat_6',
            'pass_7',	'pass_dis_7',	'pass_type_7',	'pass_mat_7',
            'pass_8',	'pass_dis_8',	'pass_type_8',	'pass_mat_8',
            'signal_start']
TestingData=pd.DataFrame(data=X_test, columns=Predictors)
TestingData['Loss']=y_test[:,0]
TestingData['Predicted_Loss']=pred[:,0]
TestingData['Error_Loss']=abs(y_test[:,0]-pred[:,0])/(y_test[:,0])*100
TestingData['Signal']=y_test[:,1]
TestingData['Predicted_Signal']=pred[:,1]
TestingData['Error_Signal']=abs(y_test[:,1]-pred[:,1])/(y_test[:,1])*100
TestingData.head(10)

In [ ]:
pred


In [ ]:
data2.head()

In [ ]:

data2.head()

In [ ]:


data2

In [ ]:


d2

In [ ]:
a=np.setdiff1d(data2.columns,X_train.columns)

a

In [ ]:
pred = model.predict(data2)
pred

In [ ]:
Predictors=['router_x',	'router_y',	'router_z',	'cell_x',	'cell_y',	'cell_z',	'Distance',	'orientation_x',	'orientation_y',	'orientation_z',
            'pass_0',	'pass_dis_0',	'pass_type_0',	'pass_mat_0',
            'pass_1',	'pass_dis_1',	'pass_type_1',	'pass_mat_1',
            'pass_2',	'pass_dis_2',	'pass_type_2',	'pass_mat_2',
            'pass_3',	'pass_dis_3',	'pass_type_3',	'pass_mat_3',
            'pass_4',	'pass_dis_4',	'pass_type_4',	'pass_mat_4',
            'pass_5',	'pass_dis_5',	'pass_type_5',	'pass_mat_5',
            'pass_6',	'pass_dis_6',	'pass_type_6',	'pass_mat_6',
            'pass_7',	'pass_dis_7',	'pass_type_7',	'pass_mat_7',
            'pass_8',	'pass_dis_8',	'pass_type_8',	'pass_mat_8',
            'signal_start']
TestingData=pd.DataFrame(data=data2, columns=Predictors)
TestingData['Predicted_Loss']=pred
TestingData.head()

In [ ]:
fig=plt.figure(figsize=(10,7))
ax=plt.axes(projection="3d")
ax.scatter(data2['router_x'],data2['router_y'],data2['router_z'])
data2['router_x']

In [ ]:
data2['router_x']

In [ ]:
x_test